In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype

# Machine Learning Libraries
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder #LabelEncoder #OneHotEncoder
from xgboost import XGBRegressor, plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import mutual_info_regression
from scipy import stats
from math import ceil
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

NameError: name 'os' is not defined

In [ ]:
Xy= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv",index_col='Id')

X_test= pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv",index_col='Id')

Xy = Xy.dropna(subset=['SalePrice'],axis=0)
# Separate target from predictors
X = Xy.drop(['SalePrice'], axis=1)
y = Xy.SalePrice


In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

In [ ]:
info = pd.DataFrame(data=X.dtypes,columns=['Dtype'])
info["Unique"]= X.nunique()
info["Null"]  = X.isnull().sum()
info

In [ ]:
y.describe()


In [ ]:
corr_matrix = X.corr()
mask = np.triu(corr_matrix.corr())
sbn.set(font_scale=1.1)
plt.figure(figsize=(25,20))
sbn.heatmap(corr_matrix,mask=mask,fmt ='.1f',annot=True,cmap='coolwarm',square=True,linewidths=2,cbar=False)
plt.show()

In [ ]:
def show_null_values(x,y):
    null_vals = pd.DataFrame(x.isnull().sum(),columns=['Train/null'])
    null_vals['Test']= y.isnull().sum()
    null_vals= null_vals.loc[(null_vals['Train/null']!=0) | (null_vals['Test']!=0) ]
    null_vals = null_vals.sort_values(by = ['Train/null','Test'],ascending=False)
    print("total null values",null_vals.sum(),sep='\n')
    return

In [ ]:
show_null_values(X,X_test)

In [ ]:
null_cols = [col for col in X.columns if X[col].isnull().sum()>len(X[col])/2]
null_cols

In [ ]:

X.drop(null_cols,axis=1,inplace=True)
X_test.drop(null_cols,axis=1,inplace=True)


In [ ]:
print(len(X.columns),len(X_test.columns))

In [ ]:
X.columns

In [ ]:
object_cols = X.select_dtypes(include="object")
len(object_cols.columns)

In [ ]:
sbn.set_style('white')
fig ,ax = plt.subplots(nrows=13,ncols=3,figsize=(25,70),sharey=True,dpi=120)
for (col,subplot) in zip(object_cols,ax.flatten()):
    #print(X[col].value_counts())
    #print(subplot)
    plt.ylim([0,800000])
    for tick in subplot.get_xticklabels():
        tick.set_rotation(45)
    sbn.violinplot(data=X,x=col,y=y,ax=subplot)

In [ ]:
print(len(X.columns))
X.drop("Utilities",axis=1,inplace=True)
print(len(X.columns))

In [ ]:

print(len(X_test.columns))
X_test.drop("Utilities",axis=1,inplace=True)
print(len(X_test.columns))

In [ ]:
x1= pd.DataFrame([[1,2],[3,4]])
x2=pd.DataFrame([[5,6],[6,7]])
#x3=pd.concat(x1,x2).................
type(x1),type(x2)
x3 = pd.concat([x1,x2],axis=1)
x3

In [ ]:
print(X.shape,X_test.shape)
df= pd.concat([X,X_test])
print(f"shape after concatination {df.shape}",type(df))

In [ ]:
X.head(3)

# df.columns

In [ ]:
df1= pd.DataFrame()
df1["Age"] = df['YrSold']-df['YearBuilt']
df1['AgeRemodel'] =df['YrSold']-df['YearRemodAdd']  
years = ['YrSold','YearBuilt','AgeRemodel','Age']
df_1=pd.concat([df,df1],axis=1).loc[:,cols]

X_1 = df_1.loc[X.index, :]
print(X_1.shape)
print(y.shape)
df_1.sample()


In [ ]:
sbn.set_style('whitegrid')
sbn.set(font_scale=1.5)
fig, ax = plt.subplots(1, 4, figsize=(20, 6), dpi=200)
for col,i in zip(cols, [0,1,2,3]):
    sbn.scatterplot(x=X_1.loc[:,col],y=y, ax=ax[i],hue=X.ExterQual, palette='pastel')
plt.ylim([0,800000])    
fig.tight_layout()
fig.text(0.5, 1, 'Distribution of SalesPrice with respect to years columns', size=40, ha="center", va="center")
plt.show()

In [ ]:
sbn.set_style("dark")
sbn.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":20},style="white")
sbn.set(font_scale=1.5)
fig,ax = plt.subplots(nrows=1,ncols=4,figsize=(20,5),sharey=True,dpi=140)
for (col,i) in zip(cols,[0,1,2,3]):
    sbn.scatterplot(x=X_1.loc[:,col],y=y,hue=X.ExterQual, palette='pastel',ax=ax[i])
fig.tight_layout()
fig.text(0.5,1,"Scatter Plot For various columns",size=30,ha='center')
plt.show()

finding corelation with years columns with sale price

In [ ]:
X_1.corrwith(y)

In [ ]:
df2 = pd.DataFrame()
df2['Remodel']= df['YearRemodAdd']!=df['YearBuilt']
df2["Garage"] = df['GarageQual'].notnull()
df2['FirePlace'] = df['FireplaceQu'].notnull()
df2["Bsmt"]   =  df["BsmtQual"].notnull()
df2['Masonry'] = df['MasVnrType'].notnull()
df2.replace([True,False],[1,0],inplace=True)
print(df2.shape)
df2.head()

In [ ]:
df.shape

In [ ]:
object_cols = df.select_dtypes(include=['object']).columns
ordinal_col1 = [col for col in object_cols if ("QC" in col)or ("Qu" in col)or  ("Fin" in col) or("Cond" in col)  and ('Condition' not in col)]
{col:[*df[col].unique()] for col in ordinal_col1}

In [ ]:
object_cols = df.select_dtypes(include=['object']).columns
ordinal_cols1 = [col for col in object_cols if ("QC" in col)or ("Qu" in col)or  ("Fin" in col) or("Cond" in col)  and ('Condition' not in col)]
print(ordinal_cols1)
6#print({col:[*df[col].unique()] for col in ordinal_col1})
df.loc[:,ordinal_col1] = df.loc[:,ordinal_col1].fillna("NA")
df.loc[:,ordinal_col1] = df.loc[:,ordinal_col1].replace(['NA','Fa','Po','TA','Gd','Ex'],[0,1,2,3,4,5])
ordinal_col2 = ['BsmtFinType1','BsmtFinType2']
df.loc[:,ordinal_col2] =  df.loc[:,ordinal_col2].fillna('NA')
df.loc[:,ordinal_col2] =  df.loc[:,ordinal_col2].replace(['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], [0,1,2,3,4,5,6])
ordinal_cols3 = ['BsmtExposure']
df.loc[:,ordinal_cols3] = df.loc[:,ordinal_cols3].fillna('NA')
df.loc[:,ordinal_cols3] = df.loc[:,ordinal_cols3].replace(['NA', 'No', 'Mn', 'Av', 'Gd'], [0,1,2,3,4])
ordinal_cols4 = ['LotShape']
df.loc[:,ordinal_cols4] = df.loc[:,ordinal_cols4].replace(['Reg', 'IR1', 'IR2', 'IR3'], [0,1,2,3])
ordinal_cols5 = ['GarageFinish']
df.loc[:,ordinal_cols5] = df.loc[:,ordinal_cols5].replace(['NA', 'Unf', 'RFn', 'Fin'], [0,1,2,3])
ordinal_cols6 = ['Functional']
df.loc[:,ordinal_cols3] = df.loc[:,ordinal_cols3].fillna('Mod')
df.loc[:,ordinal_cols6] = df.loc[:,ordinal_cols6].replace(["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"], list(range(8)))
df.shape

In [ ]:
o_columns = ordinal_cols1+ordinal_col2+ordinal_cols3+ordinal_cols4+ordinal_cols5+ordinal_cols6
df.loc[:,o_columns].dtypes.value_counts()

In [ ]:
Bath_cols = [i for i in df.columns if 'Bath' in i]
Bath_cols

In [ ]:
SF_cols = ['TotalBsmtSF','1stFlrSF','2ndFlrSF']
df[SF_cols+Bath_cols] = df[SF_cols+Bath_cols].fillna(0)

In [ ]:
df3 = pd.DataFrame() 
df3["Liv_Qual"] = (df.OverallQual + df.OverallCond/3) * df.GrLivArea
df3["GarageArea_Qual"] = (df.GarageQual + df.GarageCond/3) * df.GarageArea * df.GarageCars
df3['BsmtArea_Qual'] = (df.BsmtQual * df.BsmtCond/3) * df.TotalBsmtSF
df3["LivLotRatio"] = df.GrLivArea / df.LotArea
df3["Spaciousness"] = (df['1stFlrSF'] + df['2ndFlrSF']) / df.TotRmsAbvGrd
df3['TotalSF'] = df[SF_cols].sum(axis = 1)
df3['TotalBath'] = df.FullBath + df.BsmtFullBath + df.HalfBath/2 + df.BsmtHalfBath/2
df3.head()

In [ ]:
df4 = pd.DataFrame()

Porches = ["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"]
df4["PorchTypes"] = df[Porches].gt(0.0).sum(axis=1)
df4.head()


In [ ]:
df5 = pd.DataFrame()
df5["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")

In [ ]:


df6 = pd.DataFrame()  # dataframe to hold new features

df6 = pd.get_dummies(df.BldgType, prefix="Bldg")
df6 = df6.mul(df.GrLivArea, axis=0)



In [ ]:
before = df.shape[1]

# Drop categorical columns with high cardinality (number of unique values in a column)
cat_columns_to_drop = [cname for cname in X.select_dtypes(["object","category","bool"]).columns
                    if X[cname].nunique() > 10]
df.drop(cat_columns_to_drop, axis=1, inplace=True)

after = df.shape[1]

print(f'Number of columns reduced from {before} to {after}')

In [ ]:
cat_columns = list(df.select_dtypes('object').columns)
before = df[cat_columns].nunique().sum()
before

In [ ]:
df.HouseStyle.value_counts(normalize=True)

In [ ]:
for col in cat_columns:
    df[col]=df[col].mask(df[col].map(df[col].value_counts(normalize=True)) < 0.01, 'Other')

In [ ]:
after = df[cat_columns].nunique().sum()
print(f'Number of unique categories reduced from {before} to {after}')

In [ ]:
df.HouseStyle.value_counts(normalize=True)*100

In [ ]:
features_nom = ["MSSubClass"] + cat_columns

# Cast each of the above 21 columns into 'category' DataType
for name in features_nom:
    df[name] = df[name].astype("category")
    
    # Add a None category for missing values
    if "NA" not in df[name].cat.categories:
        df[name] = df[name].cat.add_categories("NA")

In [ ]:
for colname in df.select_dtypes(["category"]):
    df[colname] = df[colname].cat.codes

In [ ]:
df.dtypes.value_counts()

In [ ]:



df = pd.concat([df,df1,df2,df3,df4,df5,df6], axis=1)
df.drop(years+Porches,axis=1, inplace=True)
df.sample()



In [ ]:
X = df.loc[X.index, :]
X_test = df.loc[X_test.index, :]

In [ ]:
print(X.shape,X_test.shape,sep='\n')

In [ ]:
my_imputer = SimpleImputer(missing_values = np.nan, strategy ='constant', fill_value=0)
 
# Fitting the data to the imputer object
imputed_X = pd.DataFrame(my_imputer.fit_transform(X))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

# Imputation removed column names and indices; put them back
imputed_X.columns = X.columns
imputed_X_test.columns = X_test.columns
imputed_X.index = X.index
imputed_X_test.index = X_test.index
 
# Using original names of DataSets
X = imputed_X
X_test = imputed_X_test

In [ ]:
show_null_values(X, X_test)

In [ ]:


X_y = X.copy()
X_y['SalesPrice'] = y
X_y.sample()



In [ ]:
def univariate_numerical_plot(df, x):
    fig, ax = plt.subplots(1, 3, figsize=(20, 6), dpi=100)
    
    # 0) histogram
    sbn.histplot(data=df, x=x, kde=True, ax=ax[0], bins=min(df[x].nunique(),10), kde_kws={'bw_adjust':3})
    sbn.despine(bottom=True, left=True)
    ax[0].set_title('histogram')
    ax[0].set_xlabel(xlabel=x)
    
    # 1) box plot
    sbn.boxplot(data=df, x=x, ax=ax[1])
    ax[1].set_title('boxplot')
    ax[1].set_ylabel(ylabel=x)
    
    # 2) scatterplot
    sbn.scatterplot(x=df[x], y=y, ax=ax[2], hue=y ,palette='coolwarm')
    plt.legend([],[], frameon=False)
    
    # To add border
    fig.subplots_adjust(top=0.85, bottom=0.15, left=0.2, hspace=0.8)
    fig.patch.set_linewidth(10)
    fig.patch.set_edgecolor('cornflowerblue')
    
    fig.tight_layout()
    fig.text(0.5, 1, f'Distribution of {x}', size=25, ha="center", va="center")
    plt.show()

In [ ]:
univariate_numerical_plot(X_y,'SalesPrice')

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    # All discrete features should now have integer dtypes
    # discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X.select_dtypes('number'), y, random_state=0)
    mi_scores = pd.DataFrame(mi_scores.round(2), columns=["MI_Scores"], index=X.select_dtypes('number').columns)
    return mi_scores

In [ ]:
mi_scores = make_mi_scores(X, y)
linear_corr = pd.DataFrame(X.corrwith(y).round(2), columns=['Lin_Correlation'])

corr_with_price = pd.concat([mi_scores, linear_corr], axis=1)
corr_with_price = corr_with_price.sort_values('MI_Scores',ascending=False)

corr_with_price

In [ ]:
top_features = corr_with_price.index[1:6]

In [ ]:
for feature in top_features:
    univariate_numerical_plot(X,feature)

In [ ]:
threshold = 0.01
numerical_cols = [cname for cname in X.select_dtypes('number').columns
                  if corr_with_price.MI_Scores[cname] > threshold]

# Keep selected columns only
selected_cols = numerical_cols
X = X[selected_cols]
X_test = X_test[selected_cols]
after = X.shape[1]

# Selected Features for Model Training or Fitting
print(f'Out of {before} features, {after} fetures are having MI_Scores more than {threshold}.')

In [ ]:
info = pd.DataFrame(X.dtypes, columns=['Dtype'])
info['Unique'] = X.nunique().values
info['Null'] = X.isnull().sum().values
info.sort_values(['Dtype', 'Unique'])

In [ ]:
xgb = XGBRegressor(eval_metric='rmse')

In [ ]:
param_grid = [
    {'subsample': [0.5], 'n_estimators': [1400], 
     'max_depth': [5], 'learning_rate': [0.02],
     'colsample_bytree': [0.4], 'colsample_bylevel': [0.5],
     'reg_alpha':[1], 'reg_lambda': [1], 'min_child_weight':[2]}
]
grid_search = GridSearchCV(xgb, param_grid, cv=3, verbose=1, scoring='neg_root_mean_squared_error')

In [ ]:
grid_search.fit(X, np.log(y));

In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)
display(cv_results.sort_values('rank_test_score')[:6])

In [ ]:
grid_search.best_params_

In [ ]:
print("RMSLE on training data:",round(-grid_search.score(X, np.log(y)),4))
print("RMSLE on validation data:",round(-grid_search.best_score_,4))

In [ ]:
Feature_Imp = grid_search.best_estimator_.feature_importances_
Feature_Imp_sorted_series = pd.Series(Feature_Imp,X.columns).sort_values(ascending=True)

# Plot horizaontal bar plot
plt.figure(figsize=(14,20), dpi=120)
palette = sns.color_palette("coolwarm", len(X.columns)).as_hex()
ax = Feature_Imp_sorted_series.plot.barh(width=0.8 ,color=palette)
ax.set_title('Feature Importance in XgBoost')
plt.show()

In [ ]:
y_preds = np.exp(grid_search.predict(X_test))

In [ ]:


output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': y_preds.round()})
output.to_csv('submission.csv', index=False)

